In [ ]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.double_sorting import calculate_period_return
from environ.process.asset_pricing.fama_macbeth import build_dominance_factor, calculate_weekly_returns

def reg_fama_macbeth(data_fama_macbeth, formula:str = "dollar_ret ~ DOM"
) -> pd.DataFrame:
  risk_premia = (data_fama_macbeth
    .groupby(["Year", "Week"])
    .apply(lambda x: sm.ols(
        formula= formula, 
        data=x
      ).fit()
      .params
    )
    .reset_index()
  )

  price_of_risk = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")["estimate"]
    .apply(lambda x: pd.Series({
        "risk_premium": x.mean(),
        "t_stat": x.mean()/x.std()*np.sqrt(len(x))
      })
    )
    .reset_index()
    .pivot(index="factor", columns="level_1", values="estimate")
    .reset_index()
  )

  price_of_risk_NW = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")
    .apply(lambda x: (
        x["estimate"].mean()/ 
          sm.ols("estimate ~ 1", x)
          .fit(cov_type="HAC", cov_kwds={"maxlags": 4}).bse
      )
    )
    .reset_index()
    .rename(columns={"Intercept": "t_stat_NW"})
  )
  # express risk premium in percentage
  price_of_risk['risk_premium'] = price_of_risk['risk_premium'] * 100
  return price_of_risk.merge(price_of_risk_NW, on="factor").round(3)

In [11]:
# reg_panel = pd.read_pickle(
#     PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
# )
# reg_panel.columns

In [12]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

reg_panel["daily_supply_return"] = reg_panel["supply_rates"] / 365.2425


def get_weekly_panel(reg_panel):
    ### Get test assets returns 
    reg_panel['Date'] = pd.to_datetime(reg_panel['Date'])
    reg_panel['Year'] = reg_panel['Date'].dt.year
    reg_panel['Week'] = reg_panel['Date'].dt.isocalendar().week

    # add daily simple returns and convenience yield
    reg_panel = calculate_period_return(df_panel=reg_panel, freq=1, simple_dollar_ret=False)

    # Change into weekly returns
    reg_panel = calculate_weekly_returns(df_panel=reg_panel)

    reg_panel['weekly_dollar_ret'] = reg_panel.groupby(['Year', 'Week'])['weekly_dollar_ret'].transform(
            lambda x: x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99))
    )
    return reg_panel
    
# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    # "all": reg_panel,
}

for panel_info, df_panel in stable_nonstable_info.items():
    for simple_dollar_ret in [True, False]:
        for dominance in DEPENDENT_VARIABLES[:2]: 
            print(f"Processing {panel_info} {dominance}")
            dominance_factor = build_dominance_factor(
                    reg_panel=df_panel,
                    brk_pt_lst=[0.8],
                    dominance_var=dominance,
                    zero_value_portfolio=False,
                    simple_dollar_ret=simple_dollar_ret,
                )
            dominance_factor.drop(['P1', 'P2'], axis=1, inplace=True)
            dominance_factor.rename(columns={"P2 - P1": "DOM"}, inplace=True)

            reg_panel = get_weekly_panel(df_panel)
            # merge all factors
            data_fama_macbeth = pd.merge(dominance_factor, ff3, on=['Year', 'Week'], how="left")
            data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=['Year', 'Week'], how="left")
            # merge factors with returns
            data_fama_macbeth = pd.merge(data_fama_macbeth, reg_panel, on=['Year', 'Week'], how="left")
            data_fama_macbeth.rename(columns={"weekly_dollar_ret": "dollar_ret", "weekly_ret":"convenience_ret"}, inplace=True)
            data_fama_macbeth = data_fama_macbeth.dropna()

            # run the Fama-Macbeth regression
            fama_macbeth = reg_fama_macbeth(data_fama_macbeth, formula="dollar_ret ~ CMKT + CMOM + CSIZE + DOM")
            display(fama_macbeth)


Processing stablecoin volume_ultimate_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.000,0.166,0.199
1,CMOM,0.001,0.813,0.856
2,CSIZE,-0.001,-1.231,-1.225
3,DOM,-0.000,-1.913,-1.942
4,Intercept,0.006,0.405,0.422


Processing stablecoin eigen_centrality_undirected


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.000,0.166,0.199
1,CMOM,0.001,0.813,0.856
2,CSIZE,-0.001,-1.231,-1.225
3,DOM,-0.000,-2.102,-1.720
4,Intercept,0.006,0.406,0.422


Processing stablecoin volume_ultimate_share


...

,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.007,5.474,5.418
1,CMOM,-0.001,-0.690,-0.655
2,CSIZE,0.002,2.856,2.238
3,DOM,-0.000,-0.129,-0.138
4,Intercept,0.016,1.515,1.133


Processing non-stablecoin volume_ultimate_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.007,5.473,5.405
1,CMOM,-0.000,-0.623,-0.588
2,CSIZE,0.002,2.880,2.247
3,DOM,-0.000,-0.134,-0.119
4,Intercept,0.016,1.469,1.100


Processing non-stablecoin eigen_centrality_undirected


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.007,5.473,5.405
1,CMOM,-0.000,-0.623,-0.588
2,CSIZE,0.002,2.880,2.247
3,DOM,0.000,0.076,0.067
4,Intercept,0.016,1.469,1.100
